In [13]:
import pyreadstat
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [15]:
#load all data

don=pd.read_csv("../1_data/private/donation_donor.csv", error_bad_lines=False)
defr=pd.read_csv("../1_data/private/deferral.csv", error_bad_lines=False)
collect=pd.read_csv("../1_data/private/collec_sites.csv", error_bad_lines=False) #

med_quest=pd.read_csv("../1_data/private/medical_quest.csv", error_bad_lines=False)

#dimensions
print(don.shape)
print(defr.shape)

In [16]:
med_questHGB=med_quest.loc[(med_quest['QUESTION_CODE']=='5U')] #df with hgb values
med_questHGB_recheck=med_quest.loc[(med_quest['QUESTION_CODE']=='5V')] #df with hgb recheck values

In [17]:
# code adopted from 00_classifying deferrals.py (refer to that for more details)
# Investiagting deferral table
# drop don date and remove duplicates
defr2=defr.drop(['DON_DATE_KEY'], axis=1)
defr2.drop_duplicates(inplace=True)
print(len(defr2))#len after shrink

#change deferral start_date to don_date_key
defr2['DON_DATE_KEY']=defr2['DEFERRAL_START_DATE'].str.slice(start=0, stop=10)
defr2['DON_DATE_KEY']=defr2['DON_DATE_KEY'].str.replace(r"\D", "") + ".0"


4666763


In [19]:
# merge don with collect

#merge donations and deferrals
df1=pd.merge(don, collect, how='left', on=["COLLECTION_SITE_KEY"])

df1.head()

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,...,DONOR_EDUCATION_DESC,DONOR_WEIGHT,DONOR_HEIGHT,DONOR_ZIPCODE,DONOR_AGE_AT_DONATION,COLLECTION_FIXED_LOCATION,COLLECTION_SITE_VALID_FLAG,BECS_ID,DRIVE_SETUP_TYPE,DRIVE_TYPE_FIXED_MOBILE
0,20130625.0,15057895.0,82X912T,W28861300166600F,Active,3003.0,62130.0,NaN,NaN,18:58:01,...,BACHELOR'S DEGREE,278.0,74.0,606,56.0,Yes,Y,VTL,Fixed,Fixed
1,20140525.0,840652.0,5006735,W04221402205100P,First Time,51.0,383.0,500.0,612.0,17:04:01,...,SOME COLLEGE OR TECHNICAL SCHOOL,175.0,69.0,890,63.0,No,Y,VTL,Mobile Setup,Mobile
2,20141018.0,792640.0,2109508,W04221404264200Q,Active,51.0,376.0,452.0,600.0,10:24:01,...,SOME COLLEGE OR TECHNICAL SCHOOL,205.0,68.0,890,36.0,Yes,Y,VTL,Fixed,Fixed
3,20140726.0,7079135.0,5024441,W04221402021200I,First Time,51.0,380.0,500.0,602.0,12:28:01,...,HIGH SCHOOL GRAD-DIPLOMA OR EQUIV,190.0,70.0,891,25.0,Yes,Y,VTL,Fixed,Fixed
4,20140519.0,6126680.0,4186033,W042214021267009,Active,51.0,383.0,440.0,612.0,11:31:01,...,"9th,10th,11th,12th GRADE-NO DIPLOMA",190.0,69.0,864,18.0,No,Y,VTL,Mobile Setup,Mobile


In [20]:
#merge df with deferrals
defr2['DON_DATE_KEY']=defr2['DON_DATE_KEY'].apply(pd.to_numeric)

df1=pd.merge(df1, defr2, how='left', on=["DONOR_KEY", "DON_DATE_KEY", 'DONATION_NUMBER'])
print(len(df1))

df1=pd.merge(df1, med_questHGB[["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], how='left', on=["DONOR_NUMBER","DONATION_NUMBER", "DON_DATE_KEY" ])

df1=pd.merge(df1, med_questHGB_recheck[["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], how='left', on=["DONOR_NUMBER", "DONATION_NUMBER", "DON_DATE_KEY"])

15897689


In [21]:
#merge hemoglobin level from med_quest
df1=pd.merge(df1, med_questHGB[["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], how='left', on=["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER"])

df1.head()

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,...,DEFERRAL,DEFERRAL_DESCRIPTION,DEFERRAL_VALID_CODE,DEFERRAL_DURATION,DURATION_MODIF_TYPE,ALLOW_DISTRIBUTION,DEFERRAL_GROUP,MQ_ANSWER_x,MQ_ANSWER_y,MQ_ANSWER
0,20130625.0,15057895.0,82X912T,W28861300166600F,Active,3003.0,62130.0,NaN,NaN,18:58:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20140525.0,840652.0,5006735,W04221402205100P,First Time,51.0,383.0,500.0,612.0,17:04:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.8,NaN,15.8
2,20141018.0,792640.0,2109508,W04221404264200Q,Active,51.0,376.0,452.0,600.0,10:24:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6,NaN,16.6
3,20140726.0,7079135.0,5024441,W04221402021200I,First Time,51.0,380.0,500.0,602.0,12:28:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.3,NaN,16.3
4,20140519.0,6126680.0,4186033,W042214021267009,Active,51.0,383.0,440.0,612.0,11:31:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.1,NaN,16.1


In [22]:
#replace hgb values with recheck values

df1=pd.merge(df1, med_questHGB_recheck[["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER", 'MQ_ANSWER']], how='left', on=["DONOR_NUMBER", "DON_DATE_KEY", "DONATION_NUMBER"])

df1.head()

,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,...,DEFERRAL_DESCRIPTION,DEFERRAL_VALID_CODE,DEFERRAL_DURATION,DURATION_MODIF_TYPE,ALLOW_DISTRIBUTION,DEFERRAL_GROUP,MQ_ANSWER_x,MQ_ANSWER_y,MQ_ANSWER_x,MQ_ANSWER_y
0,20130625.0,15057895.0,82X912T,W28861300166600F,Active,3003.0,62130.0,NaN,NaN,18:58:01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20140525.0,840652.0,5006735,W04221402205100P,First Time,51.0,383.0,500.0,612.0,17:04:01,...,NaN,NaN,NaN,NaN,NaN,NaN,15.8,NaN,15.8,NaN
2,20141018.0,792640.0,2109508,W04221404264200Q,Active,51.0,376.0,452.0,600.0,10:24:01,...,NaN,NaN,NaN,NaN,NaN,NaN,16.6,NaN,16.6,NaN
3,20140726.0,7079135.0,5024441,W04221402021200I,First Time,51.0,380.0,500.0,602.0,12:28:01,...,NaN,NaN,NaN,NaN,NaN,NaN,16.3,NaN,16.3,NaN
4,20140519.0,6126680.0,4186033,W042214021267009,Active,51.0,383.0,440.0,612.0,11:31:01,...,NaN,NaN,NaN,NaN,NaN,NaN,16.1,NaN,16.1,NaN


In [23]:
#merge drive dataset
drive=pd.read_csv("../1_data/private/drive.csv", error_bad_lines=False)
df1=pd.merge(df1, drive, how='left', on=["DRIVE_ID"])

In [24]:
#merge donor history
df1.drop(df1[df1['DON_DATE_KEY'].isna()].index, inplace=True) #remove any missing visit dates

df1["Visit_Date"] = pd.to_datetime(df1['DON_DATE_KEY'], format='%Y%m%d') #convert to datetime object

df1['DonorID']=df1['DONOR_KEY']
df1['Visit_Date']=pd.to_datetime(df1['Visit_Date']).dt.strftime('%Y-%m-%dT%H:%M:%SZ') #covert to ISO -#2019-03-26T00:00:00Z

don_hist=pd.read_csv("../1_data/private/vitalant_intermediate_to_del6.csv")
df1=pd.merge(df1, don_hist[['DONATION_NUMBER', 'DonorID', 'Visit_Date','cum_lifetime_donations','unit_rbc_loss','time_to_fu',
                          'rbc_loss_last_12_months', 'rbc_loss_last_24_months','days_since_last_rbc_loss', 
                          'days_since_last_drbc_loss']], how='left', on=["DonorID","DONATION_NUMBER", "Visit_Date"])

df1.head()


,DON_DATE_KEY,DONOR_KEY,DONOR_NUMBER,DONATION_NUMBER,ACTIVE_DONOR_TYPE,SITE_KEY,COLLECTION_SITE_KEY,DONATION_VOLUME_DRAWN,DRIVE_ID,PHLEB_START_TIME_KEY,...,SITE_ZIPCode,Visit_Date,DonorID,cum_lifetime_donations,unit_rbc_loss,time_to_fu,rbc_loss_last_12_months,rbc_loss_last_24_months,days_since_last_rbc_loss,days_since_last_drbc_loss
0,20130625.0,15057895.0,82X912T,W28861300166600F,Active,3003.0,62130.0,NaN,NaN,18:58:01,...,NaN,2013-06-25T00:00:00Z,15057895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20140525.0,840652.0,5006735,W04221402205100P,First Time,51.0,383.0,500.0,612.0,17:04:01,...,NaN,2014-05-25T00:00:00Z,840652.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20141018.0,792640.0,2109508,W04221404264200Q,Active,51.0,376.0,452.0,600.0,10:24:01,...,152,2014-10-18T00:00:00Z,792640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20140726.0,7079135.0,5024441,W04221402021200I,First Time,51.0,380.0,500.0,602.0,12:28:01,...,NaN,2014-07-26T00:00:00Z,7079135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20140519.0,6126680.0,4186033,W042214021267009,Active,51.0,383.0,440.0,612.0,11:31:01,...,NaN,2014-05-19T00:00:00Z,6126680.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#dfsub.to_csv(("../1_data/private/sub_data.csv"), index=False)
df1.to_csv('../1_data/private/VitalantData.csv', index=False)